### dbGap FHIR Notebooks - Exercise 3

## Learning Objectives and Key Concepts

In this exercise, you will query the dbGaP FHIR test study: phs002409: 


- Recreate subject data as submitted 
- Aggregate FHIR Observations for a subject
- Enable Dataframe queries 
- Explore the same queries via FHIR

## Motivation/Purpose
To explore challenges in different representations of dbGaP data

## Requires
- ipywidgets
- pandas

 ### Icons in this Guide
 📘 A link to a useful external reference related to the section the icon appears in  

 🖐 A hands-on section where you will code something or interact with the server  
 
 
Acknowledging use of code snippets from [NIH FHIR training](https://github.com/NIH-ODSS/fhir-exercises/tree/main/Python) Exercise 0.



In [1]:
import logging

from dbgapfhir import dbgapfhir

FHIR_SERVER = 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1'
mf = dbgapfhir(FHIR_SERVER)

### Query subjects for the study

First some basic exploration of the data in the study is helpful via FHIR.

Find all the patients registered as subjects to the study.

In [2]:
study_id = 'phs002409'
patients = mf.run_query(f"Patient?_has:ResearchSubject:individual:study={study_id}")

Total  Resources: 813
Total  Bytes: 365840
Time elapsed 29.5767 seconds


In [3]:
patients[812]

{'resourceType': 'Patient',
 'id': '3635999',
 'meta': {'tag': [{'system': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/CodeSystem/DbGaP-Backend-Source',
    'code': 'Nitro',
    'display': 'Resource served by Nitro backend.'}]},
 'identifier': [{'id': '3635999',
   'system': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/CodeSystem/DbGaPConcept-DbGaPSubjectIdentifier'}],
 'gender': 'unknown'}

### Look at the study details

In [4]:
studies = mf.run_query(f"ResearchStudy?_id={study_id}")

Total  Resources: 1
Total  Bytes: 4442
Time elapsed 0.1626 seconds


In [5]:
studies[0]

{'resourceType': 'ResearchStudy',
 'id': 'phs002409',
 'meta': {'versionId': '1',
  'lastUpdated': '2022-02-14T02:04:05.878-05:00',
  'source': '#aKIWh8axWfWYc89q',
  'security': [{'system': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/CodeSystem/DbGaPConcept-SecurityStudyConsent',
    'code': 'public',
    'display': 'public'}]},
 'extension': [{'url': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/StructureDefinition/ResearchStudy-StudyOverviewUrl',
   'valueUrl': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs002409.v1.p1'},
  {'url': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/StructureDefinition/ResearchStudy-ReleaseDate',
   'valueDate': '2021-09-09'},
  {'url': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/StructureDefinition/ResearchStudy-StudyConsents',
   'extension': [{'url': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/StructureDefinition/ResearchStudy-StudyConsents-StudyConsent',
     'valueCoding': {'system': 'https://dbgap-api.ncbi.nlm.nih.gov/fhir/x1/

### Observations
What else might we look at? Let's try Observations.

The query below was simply copied from the Kids First examples. It seems reasonable but is not yet implemented in dbGaP tFHIR.

Patient?_has:ResearchSubject:individual:study={study_id}&_revinclude=Observation:subject"

We use instead a workaround to run a query for the Observations for each Patient.

The following creates a dataframe showing the observations for each patient.
It also
* saves the definition of each observation (column) to a file
* checks that the definition is the same for each instance of the observation are the same.

This is also an opportunity to illustrate the use of iPython widgets to display a progress bar.

📘 You can find out more about progress bars and other iPython widgets [here](https://ipywidgets.readthedocs.io/en/stable/).

Because we are going to make a large number of requests in a short period of time this is a good example of a task where having an api key is helpful.

📘See notebook 2 for details.

In [7]:
import os
API_KEY_PATH = '~/.keys/ncbi_api_key.txt'

# the os.path.expanduser expands file paths which include ~/ representation for the user's home directory
with open(os.path.expanduser(API_KEY_PATH)) as f:  
    api_key = f.read()
    
    mf = dbgapfhir(FHIR_SERVER, api_key=api_key)

### Now we can run the query
It will take some time. The progress bar will do what progress bars do.

In [8]:
from ipywidgets import IntProgress
from IPython.display import display

prog = IntProgress(min=0, max=len(patients)) # instantiate the bar
display(prog) # display the bar

all_obs = []
patients_with_obs = []
for p in patients:
    # print(p['id'])
    obs = mf.run_query(f"Observation?subject={p['id']}", show_stats=False)
    if len(obs)>0:
            all_obs += obs
            patients_with_obs.append(p)
    prog.value += 1

print(f"{len(patients_with_obs)} patients had observations")
print(f"{len(all_obs)} total observations")


IntProgress(value=0, max=813)

813 patients had observations
41224 total observations


### Construct a dataframe

In [9]:
import pandas as pd
from collections import Counter


patient_observations_dict = {}
variable_definitions = {}
observations = []
obsCounter  = Counter()
codeCounter = Counter()
vccCounter = Counter()
printObsCounts = False
rlimit = 20
nn = 0
for r in all_obs:

    if r['resourceType'] == 'Observation':
        #print(json.dumps(r,indent=3))
        #nn+=1
        #if nn > rlimit:
        #    break
        subject_id = r['subject']['reference']
        obsCounter[subject_id] +=1
        obs_display_name = r['code']['coding'][0]['display']
        if 'valueQuantity' in r:
            value_text = r['valueQuantity']['value']
            #value_unit = r['valueQuantity']['unit']
        elif 'valueCodeableConcept' in r:
             value_text = r['valueCodeableConcept']['coding'][0]['display']
        else:
            value_text = 'unknown'
        codeCounter[obs_display_name] +=1
        #vccCounter[vcc_text] +=1
        observations.append(r)
        
        if subject_id not in patient_observations_dict:
            patient_observations_dict[subject_id] = {obs_display_name: value_text}
        else:
            patient_observations_dict[subject_id][obs_display_name] = value_text
            
#Summarize
print(f"Number of patients with observations {len(obsCounter.keys())}")

if printObsCounts:
    print("Observation count per patient")
    print(json.dumps(obsCounter, indent=3))
#print("Coding counts")
#print(json.dumps(codeCounter, indent=3))
df = pd.DataFrame.from_dict(codeCounter,  orient='index')

Number of patients with observations 813


### Create and display the Dataframe

In [10]:
pd.set_option("display.max_rows", 30, "display.max_columns", None)
patient_df = pd.DataFrame.from_dict(patient_observations_dict, orient='index')
#patient_df.fillna('', inplace=True)
display(patient_df)

,subj_id,consent_group,gender,protocol,base_vnum,case_control,case_control_peak,race,age_baseline,age_onset,htcm_baseline,WTKG_baseline,bmi_baseline,bmiz_baseline,tx_grp,cs_tx,cs_comb_tx,clinic_city,smoke,ENV_SMOKE,ENV_SMOKE_pretrial,IUS,pred_bursts_event1_month,pred_bursts_event1_week,pred_bursts_event2_month,pred_bursts_event2_week,edhos_event1_month,edhos_event1_week,edhos_event2_month,edhos_event2_week,prech_long,prech_short,PC20_baseline,LNPC20_baseline,PREFEV_baseline,PREFVC_baseline,PREFF_baseline,PREFEVPP_baseline,PREFVCPP_baseline,bdrabpct_baseline,ATOPY,eos_baseline,log10eos_baseline,IGE_baseline,LOG10IGE_baseline,ampf_baseline,totres_baseline,POS_SKINTEST_baseline,tot_pred_bursts,tot_edhos,amsym_baseline,cs_dose
Patient/3635187,10100288,1,2,101,1,2,2,1,7.2,2.0,135.5,34.6,15.02,1.17,9,0,0,2,0,0,0,0,8,17,48,17,48,207,48,207,21.60,24.29,0.198,2.13,1.24,2.85,91.0,87.0,92.0,8.0,0,900.0,2.710117,1059.0,2.08,265.714286,2.00,0,NaN,NaN,NaN,NaN
Patient/3635188,10100295,1,1,101,1,2,2,1,11.9,3.0,151.0,23.4,20.10,0.20,9,0,0,11,0,0,1,0,4,35,36,35,48,207,48,207,14.74,3.20,1.305,0.31,1.43,1.58,97.0,65.0,125.0,10.0,0,1637.0,2.677607,709.0,3.41,303.000000,3.20,1,5.0,2.0,1.055556,NaN
Patient/3635189,10100297,1,2,101,1,2,2,1,11.7,6.0,132.2,20.9,23.29,0.46,9,1,0,11,0,0,0,0,2,207,28,207,2,207,48,207,23.01,-7.10,1.939,0.91,2.41,1.60,68.0,91.0,101.0,12.0,0,70.0,2.863917,567.0,1.40,205.357143,NaN,1,NaN,NaN,0.550000,180.0
Patient/3635190,10100300,1,1,101,1,2,2,3,8.5,1.5,146.7,29.5,17.07,2.20,4,0,0,2,0,0,0,1,2,9,20,69,20,207,28,207,23.49,-15.60,1.314,-0.46,1.93,1.57,88.0,117.0,123.0,24.0,0,176.0,3.018700,390.0,2.24,243.125000,0.40,1,3.0,NaN,0.100000,NaN
Patient/3635191,10100302,1,1,101,1,2,2,1,7.5,6.0,130.2,29.5,17.68,1.89,9,1,0,2,0,1,0,0,48,207,8,207,48,86,48,207,0.76,4.60,0.173,-1.20,1.55,1.30,73.0,90.0,125.0,1.0,0,70.0,2.778874,1768.0,2.81,255.000000,0.60,1,NaN,NaN,0.400000,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient/3635995,10103334,1,1,101,1,2,2,3,10.0,4.0,147.7,37.4,17.11,1.32,8,1,0,12,0,1,0,0,4,104,48,207,2,207,48,207,19.13,3.59,0.463,1.31,2.48,3.55,64.0,112.0,103.0,25.0,0,583.0,2.477121,115.0,1.52,255.833333,4.25,1,6.0,4.0,0.166667,NaN
Patient/3635996,10103338,1,1,101,1,2,2,1,8.0,7.0,119.9,56.3,17.09,-0.34,9,0,0,11,0,1,1,0,20,9,12,207,48,86,48,207,8.99,NaN,0.381,-0.11,1.36,2.86,84.0,102.0,126.0,5.0,0,229.0,NaN,40.0,3.04,145.000000,NaN,1,3.0,NaN,0.923077,180.0
Patient/3635997,10103341,1,1,101,1,2,2,4,5.5,2.3,140.7,23.3,17.29,0.04,4,0,0,3,0,1,1,0,2,9,20,207,48,207,48,207,39.58,7.65,2.991,1.47,2.36,1.56,81.0,100.0,111.0,27.0,0,721.0,2.865104,344.0,2.63,135.000000,NaN,1,2.0,NaN,0.714286,180.0
Patient/3635998,10103346,1,1,101,1,2,2,1,5.7,2.5,121.3,26.2,16.59,0.96,4,0,0,3,0,0,0,1,4,52,48,207,48,207,48,207,17.36,-16.52,0.659,-0.12,1.78,1.21,89.0,82.0,NaN,15.0,0,422.0,3.126781,2520.0,2.52,419.181818,3.30,0,1.0,1.0,0.777778,180.0


Each Observation will represent one cell in the above table.

813 rows × 52 columns = 42276

🖐 Note that is not the same total 41224 listed at the end of our queries. You may want to explore why. NaN is not the only reason for the difference.

Save the table above to a file

🖐 subtitute the filename below as you wish

In [13]:
txt_file_path = 'results/phs002409_workaround_obs.txt'
patient_df.to_csv(txt_file_path, sep='\t')

### Query on observation value

#### Text value
Exercise to do - Formulate the query that would identify where the value of ENV_SMOKE is 'yes'

Ahead of doing that we can check by independent<sup>1</sup> means which patients we should get back from such a query.

The filter below on the DataFrame identfies 6 patients which match these criteria.

In [14]:
patient_df[patient_df.ENV_SMOKE=='1']

,subj_id,consent_group,gender,protocol,base_vnum,case_control,case_control_peak,race,age_baseline,age_onset,htcm_baseline,WTKG_baseline,bmi_baseline,bmiz_baseline,tx_grp,cs_tx,cs_comb_tx,clinic_city,smoke,ENV_SMOKE,ENV_SMOKE_pretrial,IUS,pred_bursts_event1_month,pred_bursts_event1_week,pred_bursts_event2_month,pred_bursts_event2_week,edhos_event1_month,edhos_event1_week,edhos_event2_month,edhos_event2_week,prech_long,prech_short,PC20_baseline,LNPC20_baseline,PREFEV_baseline,PREFVC_baseline,PREFF_baseline,PREFEVPP_baseline,PREFVCPP_baseline,bdrabpct_baseline,ATOPY,eos_baseline,log10eos_baseline,IGE_baseline,LOG10IGE_baseline,ampf_baseline,totres_baseline,POS_SKINTEST_baseline,tot_pred_bursts,tot_edhos,amsym_baseline,cs_dose
Patient/3635191,10100302,1,1,101,1,2,2,1,7.5,6.0,130.2,29.5,17.68,1.89,9,1,0,2,0,1,0,0,48,207,8,207,48,86,48,207,0.76,4.60,0.173,-1.20,1.55,1.30,73.0,90.0,125.0,1.0,0,70.0,2.778874,1768.0,2.81,255.000000,0.600000,1,NaN,NaN,0.400000,180.0
Patient/3635195,10100312,1,1,101,1,2,2,2,7.9,7.6,142.9,19.1,21.99,0.79,9,0,0,1,0,1,0,0,8,207,48,17,48,207,48,207,26.69,9.26,6.282,2.23,1.77,2.00,60.0,84.0,100.0,1.0,0,264.0,2.303196,123.0,2.25,252.272727,1.833333,1,1.0,NaN,0.100000,NaN
Patient/3635197,10100317,1,2,101,1,2,2,1,10.9,4.5,130.5,27.1,14.05,1.70,4,1,0,3,0,1,0,0,8,17,28,52,8,138,48,207,25.25,3.16,0.430,-0.53,1.09,1.84,87.0,98.0,111.0,6.0,1,NaN,2.699838,37.0,2.93,230.000000,0.230769,1,3.0,NaN,0.058824,NaN
Patient/3635198,10100319,1,2,101,1,2,2,1,12.1,2.0,118.2,43.0,22.42,-0.50,4,0,0,2,0,1,0,0,48,9,48,104,32,35,48,207,11.20,-8.75,12.074,1.75,1.17,2.76,77.0,59.0,94.0,10.0,0,141.0,NaN,543.0,3.09,246.666667,5.411765,1,9.0,2.0,0.379310,NaN
Patient/3635200,10100322,1,1,101,1,2,2,1,10.7,0.3,119.4,31.6,14.07,-1.07,9,1,0,10,0,1,0,0,4,207,48,52,48,207,48,207,14.86,-5.30,1.420,-2.25,1.22,2.30,87.0,106.0,106.0,9.0,1,563.0,2.892651,48.0,0.60,214.333333,0.266667,1,4.0,NaN,1.272727,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient/3635993,10103329,1,1,101,1,2,2,1,5.8,1.0,124.3,30.0,18.53,-0.16,4,1,0,2,0,1,1,0,2,9,36,35,48,207,48,207,6.90,13.76,0.719,0.80,1.89,1.12,65.0,99.0,103.0,6.0,0,105.0,3.109579,720.0,3.10,221.833333,4.133333,1,4.0,NaN,NaN,NaN
Patient/3635994,10103333,1,2,101,1,2,2,1,10.5,0.1,127.4,31.0,18.26,-0.07,4,0,0,8,0,1,1,1,8,207,48,35,2,207,48,207,28.44,10.10,1.518,-1.20,1.03,2.71,80.0,112.0,109.0,16.0,0,880.0,2.903633,493.0,1.28,155.666667,4.166667,1,10.0,NaN,0.500000,180.0
Patient/3635995,10103334,1,1,101,1,2,2,3,10.0,4.0,147.7,37.4,17.11,1.32,8,1,0,12,0,1,0,0,4,104,48,207,2,207,48,207,19.13,3.59,0.463,1.31,2.48,3.55,64.0,112.0,103.0,25.0,0,583.0,2.477121,115.0,1.52,255.833333,4.250000,1,6.0,4.0,0.166667,NaN
Patient/3635996,10103338,1,1,101,1,2,2,1,8.0,7.0,119.9,56.3,17.09,-0.34,9,0,0,11,0,1,1,0,20,9,12,207,48,86,48,207,8.99,NaN,0.381,-0.11,1.36,2.86,84.0,102.0,126.0,5.0,0,229.0,NaN,40.0,3.04,145.000000,NaN,1,3.0,NaN,0.923077,180.0


#### Numeric value
Exercise to do - write the FHIR query that would identify patients from this study where the PREFEV_baseline is less than 1.2.

Again we can use our DataFrame for an independent<sup>1</sup> test to indicate which Patients we would expect to result from such a FHIR query

In [15]:
patient_df[patient_df.PREFVC_baseline<1.2]

,subj_id,consent_group,gender,protocol,base_vnum,case_control,case_control_peak,race,age_baseline,age_onset,htcm_baseline,WTKG_baseline,bmi_baseline,bmiz_baseline,tx_grp,cs_tx,cs_comb_tx,clinic_city,smoke,ENV_SMOKE,ENV_SMOKE_pretrial,IUS,pred_bursts_event1_month,pred_bursts_event1_week,pred_bursts_event2_month,pred_bursts_event2_week,edhos_event1_month,edhos_event1_week,edhos_event2_month,edhos_event2_week,prech_long,prech_short,PC20_baseline,LNPC20_baseline,PREFEV_baseline,PREFVC_baseline,PREFF_baseline,PREFEVPP_baseline,PREFVCPP_baseline,bdrabpct_baseline,ATOPY,eos_baseline,log10eos_baseline,IGE_baseline,LOG10IGE_baseline,ampf_baseline,totres_baseline,POS_SKINTEST_baseline,tot_pred_bursts,tot_edhos,amsym_baseline,cs_dose
Patient/3635203,10100331,1,1,101,1,2,2,3,10.8,4.0,147.5,26.4,16.25,2.27,8,1,0,1,0,1,0,0,8,35,24,52,48,207,48,69,0.52,3.45,3.881,1.05,2.22,1.09,79.0,104.0,127.0,7.0,0,150.0,2.845718,5189.0,1.92,312.777778,2.444444,1,3.0,NaN,0.111111,NaN
Patient/3635214,10100357,1,2,101,1,2,2,1,11.8,0.5,121.1,23.2,18.66,-0.27,8,0,0,2,0,0,0,0,16,9,28,104,48,9,48,190,7.94,1.36,2.793,1.37,1.50,1.14,100.0,118.0,94.0,21.0,0,437.0,2.525045,180.0,3.33,250.833333,3.466667,1,NaN,NaN,0.666667,180.0
Patient/3635283,10100565,1,2,101,1,2,2,1,5.7,8.0,142.6,41.8,17.25,1.02,8,0,0,1,0,0,1,0,2,35,48,207,16,52,48,207,8.51,0.42,0.224,1.32,1.50,1.03,79.0,95.0,108.0,6.0,0,581.0,2.771587,832.0,2.55,158.500000,3.142857,1,NaN,NaN,1.241379,180.0
Patient/3635286,10100570,1,1,101,1,2,2,1,9.6,5.0,166.2,29.3,13.57,0.06,4,0,0,8,0,0,1,0,16,207,24,207,48,207,44,207,-6.85,4.03,0.565,0.11,1.57,1.17,80.0,87.0,111.0,21.0,0,598.0,2.636488,2026.0,2.57,251.666667,2.500000,1,NaN,4.0,NaN,180.0
Patient/3635288,10100573,1,1,101,1,2,2,1,7.7,2.0,171.0,66.8,18.63,1.48,8,1,0,11,0,0,1,0,8,9,16,207,28,121,48,207,22.37,NaN,0.080,2.48,1.85,1.17,82.0,93.0,105.0,17.0,1,1148.0,2.790285,226.0,3.00,238.181818,4.814815,1,3.0,1.0,NaN,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient/3635865,10102319,1,1,101,1,2,2,1,8.4,0.6,143.5,33.8,17.04,0.37,8,1,0,11,0,0,1,0,2,138,48,52,48,207,48,138,16.07,18.59,1.746,0.05,2.24,1.08,75.0,75.0,100.0,6.0,1,950.0,2.708421,2954.0,NaN,290.000000,2.000000,1,NaN,NaN,0.650000,NaN
Patient/3635900,10102395,1,1,101,1,2,2,1,12.8,1.0,104.9,34.3,18.82,-2.16,4,0,0,4,0,0,1,0,12,35,16,69,16,17,48,207,11.17,NaN,2.081,0.72,1.16,1.14,75.0,87.0,103.0,11.0,0,400.0,2.723456,8.0,3.54,178.500000,0.705882,1,4.0,1.0,0.500000,NaN
Patient/3635920,10102441,1,2,101,1,2,2,1,13.1,2.0,143.3,72.4,15.25,1.43,4,1,0,10,0,1,0,0,4,207,8,104,32,207,48,207,12.00,3.57,0.559,-0.16,1.62,1.11,79.0,75.0,121.0,19.0,0,700.0,1.908485,127.0,3.34,243.823529,NaN,1,2.0,1.0,1.222222,NaN
Patient/3635953,10102668,1,1,101,1,2,2,1,12.9,3.0,109.5,35.5,15.49,-0.16,9,0,0,2,0,1,0,0,12,207,12,86,48,207,16,207,7.11,3.31,7.643,-0.71,1.47,1.18,80.0,94.0,90.0,6.0,1,300.0,2.149219,2023.0,2.81,252.586207,4.307692,1,3.0,3.0,0.181818,NaN


<sup>1</sup> Neither of the tests above is a completely indpendent test, as the DataFrame was itself generated via the FHIR API. A better test would rely on an independent means of accessing the source data.

### Queries on specific Observation values

In [17]:
vals = mf.run_query("Observation?combo-code-value-quantity=phv00492057.v1.p1$gt1")

Total  Resources: 803
Total  Bytes: 2699998
Time elapsed 45.6975 seconds


In [18]:
vals = mf.run_query("Observation?combo-code-value-quantity=PX091601370000$gt150")

Total  Resources: 106
Total  Bytes: 512948
Time elapsed 5.4514 seconds
